## Huntington Beach Scraper

In [1]:
import sys
sys.path.append('/Projects/regionintelligenceai/map/')

In [2]:
# Import dependencies
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
from pathlib import Path
import os
from src.paths import PROCESSED_DATA_DIR, COMPANY_DATA_DIR
from src.const import HUNTINGTON_BEACH_PLANNING_OFFICE_PHONE, HUNTINGTON_BEACH_PLANNING_URL

def get_chrome_driver():
    """Initializes and returns a Selenium Chrome web driver using ChromeDriverManager."""
    options = Options()
    return webdriver.Chrome()

In [5]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os
import re
import datetime

#from src.driver_config import get_chrome_driver


In [6]:
url = "https://www.huntingtonbeachca.gov/government/departments/planning/major/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    df_list = pd.read_html(response.text)
    df = df_list[2]
else:
    print("Failed to retrieve the webpage. Status Code:", response.status_code)


C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_29032\1583016665.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(response.text)


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import requests
import json
import time
import os
import re
import datetime

#from src.driver_config import get_chrome_driver


def extract_project_info(driver):
    # Wait for the 'middle' div to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "middle"))
    )
    
    # Initialize an empty dictionary to store the project information
    project_data = {}
    # Get the 'middle' div element
    middle_div = driver.find_element(By.ID, 'middle')
    
    # Get all text within the 'middle' div to find the 'Project Summary' as it may not be in <p> tags
    middle_div_text = middle_div.text
    project_summary_match = re.search(r'Project Summary\s*(.*)', middle_div_text, re.DOTALL)
    
    # Find other details that are likely to be within <p> tags
    paragraphs = middle_div.find_elements(By.TAG_NAME, 'p')
    
    for paragraph in paragraphs:
        text = paragraph.text
        # Check for specific section headings in the paragraph text
        if 'Last Updated:' in text:
            project_data['Last Updated'] = text.split('Last Updated:', 1)[1].strip()
            print(f"Last Updated: {project_data['Last Updated']}")
        elif 'Project Planner:' in text:
            project_data['Project Planner'] = text.split('Project Planner:', 1)[1].strip()
            print(f"Project Planner: {project_data['Project Planner']}")
        elif 'Project Status:' in text:
            project_data['Project Status'] = text.split('Project Status:', 1)[1].strip()
            print(f"Project Status: {project_data['Project Status']}")

    if project_summary_match:
        project_summary_text = project_summary_match.group(1).strip()
        project_data['Project Summary'] = project_summary_text
        print(f"Project Summary: {project_summary_text}")
    else:
        print("Project Summary not found.")

    if not project_data:
        print("No project data found.")
        
    return pd.DataFrame([project_data])


def click_project_titles(driver, project_titles):
    # DataFrame to hold all project information
    all_project_data = pd.DataFrame()
    
    for title in project_titles:
        try:
            # Find the link by the text of the title
            link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.LINK_TEXT, title))
            )
            # Click the link
            link.click()
            # Extract information from the project's page
            project_df = extract_project_info(driver)
            all_project_data = pd.concat([all_project_data, project_df], ignore_index=True)
            
            # Navigate back to the main page
            driver.back()
            
            # Wait for a bit to ensure that the main page has loaded
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'middle'))
            )
            time.sleep(2)
            
        except Exception as e:
            print(f"An error occurred while trying to click on the title '{title}': {e}")
            # Log the error or handle it as per your requirement
    
    return all_project_data

# Your main code
url = "https://www.huntingtonbeachca.gov/government/departments/planning/major/"
driver = get_chrome_driver()
driver.get(url)

# Get the list of project titles here from your dataframe
# Here, create the `df` or get it from an existing source, for example, a CSV file.
# df = pd.read_csv('your_project_titles.csv') # if you have a CSV file with project titles
# For the purpose of this example, we will create a dummy dataframe with project titles.
# Make sure to replace this with your actual data retrieval method.
project_titles = df['Project Title'].tolist()

# Collect all project data
all_project_data = click_project_titles(driver, project_titles)

# Save the collected data to a CSV file
all_project_data.to_csv('project_data.csv', index=False)

driver.quit()

Project Status: Under Construction
Project Planner: Ricky Ramos
Last Updated: September 28, 2020
Project Summary: The Ascon Landfill, located at the southwest corner of Magnolia and Hamilton, operated as a landfill from 1938 to 1984.  The landfill originally received drilling waste from oil production in the Huntington Beach area, during the city’s oil boom. Industrial and oil field wastes were disposed of at the landfill until 1971. From 1971 to 1984 construction debris was disposed of at the site.  Clean-up activity is privately funded by a group of companies.  A public scoping meeting was held in April  2013, commencing the environmental review process.  A Draft EIR and proposed final remedy document (Remedial Action Plan - RAP) were released in August 2013 for a 45-day review period.  In October 2014, a Recirculated Draft EIR was made available for a 45-day review period.  In June 2015, the Department of Toxic Substances Control (DTSC) certified the Final EIR and approved the Final

In [16]:
# Set the index of the new data to match the original df
all_project_data.set_index(df.index, inplace=True)

# Join the dataframes
final_df = df.join(all_project_data)


In [18]:
final_df.columns

Index(['Project Title', 'Address', 'Applicant',
       'Select Status  Planning  Plan Check  Under Construction  Completed  All Projects  Withdrawn',
       'Project Status', 'Project Planner', 'Last Updated', 'Project Summary'],
      dtype='object')

In [19]:
final_df.rename(columns={'Project Title': 'projectName', 'Address': 'address', 'Project Planner': 'planner', 'Project Status': 'projectStatus', 'Project Summary': 'description', 'Last Updated': 'recentUpdate'}, inplace=True)
final_df.drop(columns=['Select Status  Planning  Plan Check  Under Construction  Completed  All Projects  Withdrawn'], inplace=True)
final_df

,projectName,address,Applicant,projectStatus,planner,recentUpdate,description
0,Ascon Landfill Site (see www.asconhb.com for m...,Southwest Corner of Magnolia Street and Hamilt...,California Department of Toxic Substances Control,Under Construction,Ricky Ramos,"September 28, 2020","The Ascon Landfill, located at the southwest c..."
1,Pierside Pavilion Expansion,300 Pacific Coast Hwy,Michael C. Adams,Under Construction,Jason Kelley,"September 28, 2020",To permit the expansion the existing Pierside ...
2,Magnolia Tank Farm,21845 Magnolia Street (west side of Magnolia S...,Shopoff Realty Investments,Planning,Ricky Ramos,"January 20, 2021","The applicant proposes to construct:\na 211,00..."
3,Hebrew Academy Expansion,"14401 Willow Lane; (east of Willow Ln, at the ...","Frank Kirollos, Westland Real Estate Group",Planning,Joanna Cortez,"October 07, 2020",To amend a Conditional Use Permit for the Hebr...
4,OC Water District Groundwater Replenishment Sy...,22212 Brookhurst Street,"Kathryn Demesa, OC Water District",Under Construction,Joanna Cortez,"March 02, 2023",To permit the expansion of a groundwater reple...
5,Park Avenue Residential,"16926 Park Ave, 92649 (terminus of Park Avenue...",Michael C. Adams Associates,Planning,Ricky Ramos,"September 28, 2020",Request to change the General Plan and zoning ...
6,Short-Term Rentals,Citywide,City of Huntington Beach,Planning,Tess Nguyen,"May 22, 2023",Short-term rentals have become very popular in...
7,Parkside Estates,"West side of Graham St., south of Warner Ave.,...",Shea Homes,Under Construction,Jason Kelley,"September 28, 2020","Project includes 111 single family residences,..."
8,Gisler Residential,"21141 Strathmoor Lane (west side, south of Blu...",Brookfield Residential (www.brookfieldresident...,Under Construction,Ricky Ramos,"February 13, 2023",The applicant proposes the following: 1) Gen...
9,Olson Residential,8371-8461 Talbert Ave. (northwest corner at Ne...,The Olson Company (www.olsonhomes.com),Under Construction,Ricky Ramos,"February 13, 2023",The applicant proposes the following: 1) Gen...


In [23]:
## Find planner information
url2 = "https://www.huntingtonbeachca.gov/government/departments/planning/staff/" 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url2, headers=headers)

if response.status_code == 200:
    df_list2 = pd.read_html(response.text)
    df2 = df_list2[1]
else:
    print("Failed to retrieve the webpage. Status Code:", response.status_code)
df2

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_29032\988141379.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list2 = pd.read_html(response.text)


,0,1
0,"Ursula Luna-Reynosa, Director of Community Dev...",[email protected]
1,"Jennifer Villasenor, Deputy Director of Commun...",[email protected]
2,"Kim De Coite, Administrative Assistant",[email protected]


In [24]:
from src.const import hb_planner_emails, hb_planner_names

ModuleNotFoundError: No module named 'src'

In [ ]:


class HuntingtonBeachScraper:
    current_date = datetime.now().strftime("%Y-%m-%d")

    def __init__(self, driver, url, original_df):
        self.driver = driver
        self.url = url
        self.original_df = original_df
    
    def fetch_initial_data(self):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        response = requests.get(self.url, headers=headers)

        if response.status_code == 200:
            df_list = pd.read_html(response.text)
            self.original_df = df_list[2]
        else:
            print("Failed to retrieve the webpage. Status Code:", response.status_code)
            self.original_df = pd.DataFrame()

    def extract_project_info(self):
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.ID, "middle"))
        )
        
        project_data = {}
        middle_div = self.driver.find_element(By.ID, 'middle')
        middle_div_text = middle_div.text
        project_summary_match = re.search(r'Project Summary\s*(.*)', middle_div_text, re.DOTALL)
        paragraphs = middle_div.find_elements(By.TAG_NAME, 'p')
        
        for paragraph in paragraphs:
            text = paragraph.text
            if 'Last Updated:' in text:
                project_data['recentUpdate'] = text.split('Last Updated:', 1)[1].strip()
            elif 'Project Planner:' in text:
                project_data['planner'] = text.split('Project Planner:', 1)[1].strip()
            elif 'Project Status:' in text:
                project_data['projectStatus'] = text.split('Project Status:', 1)[1].strip()

        if project_summary_match:
            project_data['description'] = project_summary_match.group(1).strip()

        return pd.DataFrame([project_data])

    def click_project_titles(self):
        all_project_data = pd.DataFrame()
        
        for title in self.original_df['Project Title'].tolist():
            try:
                link = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.LINK_TEXT, title))
                )
                link.click()
                project_df = self.extract_project_info()
                all_project_data = pd.concat([all_project_data, project_df], ignore_index=True)
                self.driver.back()
                WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'middle'))
                )
                time.sleep(2)
                
            except Exception as e:
                print(f"An error occurred while trying to click on the title '{title}': {e}")
        
        return all_project_data

    def merge_and_clean(self):
        self.driver.get(self.url)
        collected_data = self.click_project_titles()
        collected_data.set_index(self.original_df.index, inplace=True)
        final_df = self.original_df.join(collected_data)
        final_df.drop(columns=[col for col in final_df.columns if col not in ['Project Title', 'planner', 'projectStatus', 'description', 'recentUpdate']], inplace=True)
        final_df.rename(columns={'Project Title': 'projectName', 'Applicant': 'owner', 'Address': 'address', 'Project Planner': 'planner', 'Project Status': 'projectStatus', 'Project Summary': 'description', 'Last Updated': 'recentUpdate'}, inplace=True)
        final_df['city'] = 'Huntington Beach'
        final_df['phone'] = HUNTINGTON_BEACH_PLANNING_OFFICE_PHONE
        final_df['email'] = final_df['planner'].map(hb_planner_emails)
        self.final_df = final_df

    def save_to_company(self, path = None):
        if not Path: 
            path = COMPANY_DATA_DIR / 'huntingtonbeach' / f'huntington_beach_data_{HuntingtonBeachScraper.current_date}.xlsx' 
        self.final_df.to_excel(path, index=False)

    def save_to_processed(self, path = None):
        if not Path: 
            path = PROCESSED_DATA_DIR / 'huntingtonbeach' / f'huntington_beach_data_{HuntingtonBeachScraper.current_date}.xlsx' 
        self.final_df.to_excel(path, index=False)

# Example usage
def main_huntington_beach():
    url = HUNTINGTON_BEACH_PLANNING_URL
    # Example usage
    driver = get_chrome_driver()
    collector = HuntingtonBeachScraper(driver, url)
    collector.merge_and_clean()
    collector.save_to_company()  # Save without specifying a path will use the default
    collector.save_to_processed()  # Save without specifying a path will use the default
    driver.quit()
